In [84]:
#1. given four coordinates and a rotation angle, get XatTopEdge and YatLeftEdge.
chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')
tl, 
#2. add x to top edge and y at left edge to get the top-left chiroptera point.

#3. calculate the reset between the chiroptera and atl, and add it to atl. 

#4. add the rotated offset to the atl.

#5. run correlation!

In [109]:
import pandas as pd
import numpy as np
import math

#Because the image matcher matches ATL and chiroptera images from the top left,
#it is necessary to geolocate the original top left corner of the Chiroptera.
#To do this, we unrotate the original Chiroptera LiDAR (which is at an angle), gets (xmin, ymax), and re-rotate it.

def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

(570762.9061543589, 9497299.94695935)


In [7]:
def get_chirop_tl_coord(i, ANGLE):
    b1 = i * 4
    b2 = b1 + 4
    df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')
    ANGLE = np.deg2rad(ANGLE)
    center = ((min(df['x']) + max(df['x'])) / 2, (min(df['y']) + max(df['y'])) / 2)

    for index, row in df.iterrows():
        (x,y) = rotate(center, (row['x'], row['y']), ANGLE)
        row['x'], row['y'] = (x,y)

    (rx, ry) = (min(df['x']), max(df['y']))
    (urx, ury) = rotate(center, (rx, ry), -ANGLE)
    #df.to_csv('rot_debug_chirop_df.csv', index=False)
    print((urx, ury))
    return (urx, ury)

In [3]:
import geopandas as gpd

#This is critical for ensuring that the ATL segments match one another

atl_raster_ymins = []

for i in range(0, 8):
    b1 = i * 4
    b2 = b1 + 4
    gdf = gpd.read_file(f'../make_image/atl 1C/atl json/atl_{b1}_{b2}.json')
    xmin, ymin, xmax, ymax = gdf.total_bounds
    atl_raster_ymins.append(ymin)

In [36]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd

chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')

min_loc_val = [(35, 545),
 (46, 358),
 (66, 171),
 (84, 327),
 (147, 490),
 (206, 519),
 (80, 549),
 (305, 940)]

step = -0.18
ROT_C = np.arange(17.88, 17.88 + 8 * step, step)

def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def get_offset(i, min_loc_val, ANGLE):
    atl_df = pd.read_csv('unshifted_gt3r_full.csv')
    b1 = i * 4
    b2 = b1 + 4
    chirop_df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')
    x_min = min(chirop_df['x'])
    x_max = max(chirop_df['x'])
    y_min = min(chirop_df['y'])
    y_max = max(chirop_df['y'])
    ATL_max_y = atl_raster_ymins[i] + 4000
    ATL_min_y = ATL_max_y - 2000

    atl_df = atl_df[(atl_df['y'] >= ATL_min_y) & (atl_df['y'] <= ATL_max_y)]
    TL_POINT = (570762.3815780896, 9497300.033293018) #get_chirop_tl_coord(i, ANGLE)

    #revert ATL to Chirop TL
    x_max = max(atl_df['x'])
    y_max = max(atl_df['y'])

    offset = (TL_POINT[0] - x_max, TL_POINT[1] - y_max)
    atl_df['x'] = atl_df['x'] + offset[0]
    atl_df['y'] = atl_df['y'] + offset[1]

    atl_df.to_csv(f'debug_shifted_atl_{b1}_{b2}.csv', index=False)
    #At this point, the ATL has been matched to the top-left corner.
    #We now want to shift with the best fit (in original coordinates).
    #(x,y) (?)
    
    (best_x, best_y) = min_loc_val[i]
    best_fit = (best_x, -best_y)
    print(best_fit)
    best_fit = rotate((0,0), best_fit, -np.deg2rad(ANGLE))
    print(best_fit)
    atl_df['x'] = atl_df['x'] + best_fit[0]
    atl_df['y'] = atl_df['y'] + best_fit[1]
    atl_df.to_csv(f'debug_shifted_atl_{b1}_{b2}.csv', index=False)

get_offset(0, min_loc_val, ROT_C[0])

(35, -545)
(-134.01875715839373, -529.4232453620822)


In [8]:
get_offset(0, min_loc_val, ROT_C[0])

(570762.3815780896, 9497300.033293018)
(-90.78956921978838, 388.9090563626479)
